In [1]:
import io
import re
import csv
import pandas as pd
import win32com.client
from docx import Document

In [2]:
PathR = "R.docx"
PathS = "S.docx"
PathU = "U.docx"

In [3]:
prov = open('../provinsi.txt', 'r').read().lower()

In [8]:
prov = prov.split('\n')

In [9]:
prov = set(prov[:len(prov)-1])

In [10]:
docR = Document(open(PathR, 'rb'))
docS = Document(open(PathS, 'rb'))
docU = Document(open(PathU, 'rb'))

In [21]:
def buat_df(doc, prov, kategori):
    # REMOVE PROVINSI, 
    i = 0
    par = []
    for idx, para in enumerate(doc.paragraphs):
        if (para.text.lower() not in prov) and (len(para.text.strip(' \n\r\t')) != 0):
            tes = i, para
            par.append(tes)
            i = i+1
    
    # membaca baris per baris dengan nama perusahaan menjadi urutan pertama
    batas = 0
    pars = []
    title_bold = []
    for idx, para in par:
        if idx in title_bold:
            continue
        paratemp = []
        for run in para.runs:
            if run.bold:
                temptit = [run.text for run in para.runs if run.bold][0]
                cek_tit = True
                xx = idx + 1
                if(xx == len(par)):
                    cek_tit = False
                while cek_tit:
                    temptit2 = [run.text for run in par[xx][1].runs if run.bold]
                    if len(temptit2) > 0:
                        title_bold.append(xx)
                        temptit = temptit +' '+temptit2[0]
                        xx = xx + 1
                    else:
                        cek_tit = False
                paratemp.append(temptit+' [NAMA_PERUSAHAAN]')

                for x in range(batas, idx):
                    for run in par[x][1].runs:
                        if len(run.text.strip(' \n\r\t')) != 0:
                            if run.bold:
                                continue
                            else:
                                paratemp.append(par[x][1].text)
                pars.append(paratemp)
                batas = xx
    # menggabungkan alamat
    par_temp = []
    for idx, p in enumerate(pars):
        parss = []
        batas = len(p)-1
        batas_bawah = []
        for idxx, x in enumerate(p):
            if idxx in batas_bawah:
                continue
            tempAlamat = ''
            if idxx <= batas:
                if re.search(r'( \[NAMA_PERUSAHAAN\])', x):
                    parss.append(x)
                elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                    if idxx == len(p)-1:
                        tempAlamat = x
                    else:
                        xxx = idxx
                        while (xxx <= len(p)-1):
                            if (re.search(r'\b[\d]{5}\b', p[xxx])):
                                batas_bawah.append(xxx)
                                tempAlamat = tempAlamat + ' '+ p[xxx]
                                xxx = xxx + 1
                            elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                                batas_bawah.append(xxx)
                                tempAlamat = tempAlamat + ' '+ p[xxx]
                                xxx = xxx + 1
                    parss.append(tempAlamat.strip(' \n\r\t'))
                else:
                    parss.append(x)
        par_temp.append(parss)
    # Buat DF
    data = []
    tempTel1 = ''
    tempTel2 = ''
    tempTel3 = ''  
    tempEmail = ''
    tempWeb = ''
    for line in par_temp:

        row = {'Kategori': kategori,
               'Telp1': '', 
               'Telp2': '', 
               'Telp3': '',
               'Email': '',
               'Website': '',
               'Alamat': '',
               'Nama Perusahaan': '',
               'Kode Kategori': '',
               'Status Badan Usaha': ''}

        for text in line:
            if(re.search(r'[0-9]{3,}-\d{5,}|^08|\+62|\d{3}-62-\d{2,3}-\d{5,}|--\d{5,}|\d{6,}', text)):
                if(tempTel1 == ''):
                    tempTel1 = text
                    row['Telp1'] = tempTel1
                elif(tempTel2 == ''):
                    tempTel2 = text
                    row['Telp2'] = tempTel2
                else:
                    tempTel3 = text
                    row['Telp3'] = tempTel3
            elif(re.search(r'( \[NAMA_PERUSAHAAN\])', text)):
    #             print(text)
                text = re.sub(r'( \[NAMA_PERUSAHAAN\])', '', text)
                if (re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text)):
                    row['Nama Perusahaan'] = re.sub(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)','',text)
                    row['Status Badan Usaha'] = re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text).group().replace(',','').strip(' \n\r\t')
                else:
                    row['Nama Perusahaan'] = text
                    if(re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bGMBH\b)|(\bBUT\b)|(\bS\.P\.A\b)',text)):
                        row['Status Badan Usaha'] = re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bBUT\b)|(\bGMBH\b)|(\bS\.P\.A\b)',text).group()
                    elif(re.search(r'\bLIMITED\b',text)):
                        row['Status Badan Usaha'] = 'LTD'
                    elif(re.search(r'\b KAB \b|\b KOTA \b',text)):
                        row['Status Badan Usaha'] = 'PD'
                    else:
                        row['Status Badan Usaha'] = 'None'
            elif(re.search(r'.*@.*\.[a-zA-Z]{2,3}\.?', text)) or (re.search(r'.*@.*', text)):
                tempEmail = text
                row['Email'] = tempEmail
            elif(re.search(r'(^www.)|(^http)|(.com$)|(.co.id$)|(.net$)', text)):
                tempWeb = text
                row['Website'] = tempWeb
            elif(re.search(r'^[0-9]{2}[a-zA-Z\s\-]+( - .*)?$', text)):
                row['Kategori'] = re.sub(r'\d+ | - .*','', text)
                row['Kode Kategori'] = re.sub(r' [a-zA-Z].*','', text)
            elif(re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', text)):
                row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
            else:
                row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
        data.append(row)
        tempTel1 = ''
        tempTel2 = ''
        tempTel3 = ''  
        tempEmail = ''
        tempWeb = ''
        
    return data

In [13]:
dfR = pd.DataFrame(buat_df(docR, prov, 'Kesenian, Hiburan, dan Rekreasi'))

In [22]:
dfS = pd.DataFrame(buat_df(docS, prov, 'Aktivitas Jasa Lainnya'))

In [15]:
dfU = pd.DataFrame(buat_df(docU, prov, 'Aktivitas Badan Internasional dan Badan Ekstra Internasional Lainnya'))

In [23]:
df_gabungan = pd.concat([dfR, dfS, dfU])

In [24]:
df_gabungan.reset_index(inplace=True)

In [25]:
df_gabungan.drop(['index'], axis=1, inplace=True)

In [26]:
df_gabungan.to_csv('RSU.csv')

In [28]:
df_gabungan.tail()

,Alamat,Email,Kategori,Kode Kategori,Nama Perusahaan,Status Badan Usaha,Telp1,Telp2,Telp3,Website
7129,"KARET, KEC. SETIA BUDI, KOTA JAKARTA SELATAN",,Aktivitas Badan Internasional dan Badan Ekstra...,,UNDP REDDT,None,021-2500811,021-2500822,,
7130,"MENARA THAMRIN LT 14, JL MH THAMRIN KAV 3, KEB...",usindojkt@usindo.org,Aktivitas Badan Internasional dan Badan Ekstra...,,UNITED STATES INDONESIA SOCIETY,None,021-39830061,021-39830062,,www.usindo.org
7131,"WISMA GKBI LT 12, JL JENDRAL SUDIRMAN RT. 13/R...",,Aktivitas Badan Internasional dan Badan Ekstra...,,USAID,None,,,,
7132,"GRAHA STR, JL AMPERA RAYA RT. 02/RW. 02, RAGUN...",,Aktivitas Badan Internasional dan Badan Ekstra...,,WINROCK INTERNATIONAL <LSM>,None,021-7806884,,,
7133,"WISMA KEIAI LT 9, JL JEND SUDIRMAN KAV 3, KARE...",,Aktivitas Badan Internasional dan Badan Ekstra...,,WORLD FOOD PROGRAMME <WEP>,None,,,,
